# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


# SOLUTION

In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-07 17:46:50,840 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
import re

In [3]:
# load data

clicks = se.read.csv("hdfs:/data/clickstream.csv", sep = "\t", header=True) 
clicks.registerTempTable("clicks")
clicks.limit(5).toPandas()

,user_id,session_id,event_type,event_page,timestamp
0,562,507,page,main,1695584127
1,562,507,event,main,1695584134
2,562,507,event,main,1695584144
3,562,507,event,main,1695584147
4,562,507,wNaxLlerrorU,main,1695584154


### A small note about differences in results

The results from SQL, RDD and DF solutions are generally the same, but show +-1 difference in numbers. This happens due to non-error events having the same timestamp and different ordering may produce different routes.. 

For example: 

In [4]:
se.sql("""
SELECT *
FROM clicks AS c
WHERE user_id == 1066 AND session_id == 912;
""").show(20)

+-------+----------+----------+----------+----------+
|user_id|session_id|event_type|event_page| timestamp|
+-------+----------+----------+----------+----------+
|   1066|       912|      page|      main|1696616889|
|   1066|       912|     event|      main|1696616894|
|   1066|       912|     event|      main|1696616902|
|   1066|       912|     event|      main|1696616910|
|   1066|       912|      page|    rabota|1696616919|
|   1066|       912|      page|     bonus|1696616919|
|   1066|       912|     event|     bonus|1696616922|
|   1066|       912|     event|     bonus|1696616928|
|   1066|       912|      page|    online|1696616935|
|   1066|       912|      page|   archive|1696616939|
|   1066|       912|     event|   archive|1696616949|
|   1066|       912|     event|   archive|1696616955|
|   1066|       912|      page|    online|1696616957|
|   1066|       912|     event|    online|1696616962|
+-------+----------+----------+----------+----------+



Here, there are two events at timestamp 1696616919, rabota and bonus. Depending on how the ordering is performed, the may create a subroute main - rabota - bonus or main - bonus - rabota - bonus.

## SQL Solution

From bottom to top approach:  
1. Begin by creating a table that has lagged events - we will use it later to remove repeated events on the same page

In [5]:
# check - do all routes start with "main" event?
# it appears that no

se.sql("""
SELECT 
    event_page,
    COUNT(*)
FROM (
    SELECT *
    FROM clicks as l
    LEFT JOIN (
        SELECT 
            c.user_id, 
            c.session_id,
            MIN(c.timestamp) AS minstamp,
            1 AS mark
        FROM clicks AS c
        GROUP BY c.user_id, c.session_id
        ORDER BY c.user_id, c.session_id, 3
    ) AS r
    ON l.user_id = r.user_id
        AND l.session_id = r.session_id
        AND l.timestamp = r.minstamp
    WHERE minstamp IS NOT NULL
    ORDER BY l.user_id, l.session_id, l.timestamp) AS tbl1
GROUP BY event_page;
""").show(300)

+----------+--------+
|event_page|count(1)|
+----------+--------+
|      news|       7|
|      main|   48596|
|    rabota|       8|
|   archive|       2|
|     bonus|       5|
|    online|       3|
|  internet|       2|
|   digital|       1|
+----------+--------+



In [6]:
# check - do all routes start with "page" event?
# it appears that no, so we need to use LAG to determine change of event page

se.sql("""
SELECT 
    event_type,
    COUNT(*)
FROM (
    SELECT *
    FROM clicks as l
    LEFT JOIN (
        SELECT 
            c.user_id, 
            c.session_id,
            MIN(c.timestamp) AS minstamp,
            1 AS mark
        FROM clicks AS c
        GROUP BY c.user_id, c.session_id
        ORDER BY c.user_id, c.session_id, 3
    ) AS r
    ON l.user_id = r.user_id
        AND l.session_id = r.session_id
        AND l.timestamp = r.minstamp
    WHERE minstamp IS NOT NULL
    ORDER BY l.user_id, l.session_id, l.timestamp) AS tbl1
GROUP BY event_type;
""").show(300)

+----------+--------+
|event_type|count(1)|
+----------+--------+
|      page|   48551|
|     event|      72|
|   SerrorN|       1|
+----------+--------+



In [7]:
# Self-joined table

se.sql("""
SELECT
    *,
    LAG(c.event_page, 1, 0) OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp ASC) AS lagged_event
FROM clicks AS c
ORDER BY c.user_id, c.session_id, c.timestamp
""").show(10)

+-------+----------+----------+----------+----------+------------+
|user_id|session_id|event_type|event_page| timestamp|lagged_event|
+-------+----------+----------+----------+----------+------------+
|      0|       874|      page|      main|1696371064|           0|
|      0|       874|     event|      main|1696372696|        main|
|      0|       874|     event|      main|1696373564|        main|
|      0|       874|      page|    rabota|1696374894|        main|
|      0|       874|     event|    rabota|1696377393|      rabota|
|      0|       874|      page|    online|1696378229|      rabota|
|      0|       874|     event|    online|1696378928|      online|
|      0|       879|      page|      main|1696768416|           0|
|      0|       879|      page|    online|1696768738|        main|
|      0|       879|      page|   tariffs|1696768973|      online|
+-------+----------+----------+----------+----------+------------+
only showing top 10 rows



2. Next, we need a table of errors: find rows where there is `error` text in the event type

In [8]:
# Errors

se.sql("""
SELECT user_id, session_id, timestamp, 1 AS is_error
FROM clicks
WHERE event_type LIKE "%error%"
""").show(10)

+-------+----------+----------+--------+
|user_id|session_id| timestamp|is_error|
+-------+----------+----------+--------+
|    562|       507|1695584154|       1|
|   4567|       514|1695584351|       1|
|    461|       174|1695584529|       1|
|    844|       258|1695584652|       1|
|    461|       174|1695584698|       1|
|    805|       626|1695585321|       1|
|   4567|       514|1695586124|       1|
|   3972|        31|1695586342|       1|
|    512|       781|1695586400|       1|
|    512|       781|1695586562|       1|
+-------+----------+----------+--------+
only showing top 10 rows



3. Next we semi-join the two tables, so that all timestamps higher than the error timestamp within user-id and session-id keys are marked as errors.

In [9]:
# Timestamps after errors

se.sql("""
SELECT DISTINCT
    c.user_id,
    c.session_id,
    c.event_type,
    c.event_page,
    c.timestamp,
    c.lagged_event,
    e.is_error
FROM (
    SELECT DISTINCT
        *,
        LAG(c.event_page, 1, 0) OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp ASC) AS lagged_event
    FROM clicks AS c
    ORDER BY c.user_id, c.session_id, c.timestamp) 
AS c
LEFT JOIN (
    SELECT DISTINCT user_id, session_id, timestamp, 1 AS is_error
    FROM clicks
    WHERE event_type LIKE "%error%"
    ) AS e
ON
    c.user_id = e.user_id
    AND c.session_id = e.session_id
    AND c.timestamp >= e.timestamp
ORDER BY c.user_id, c.session_id, c.timestamp
""").show(30)

+-------+----------+----------+----------+----------+------------+--------+
|user_id|session_id|event_type|event_page| timestamp|lagged_event|is_error|
+-------+----------+----------+----------+----------+------------+--------+
|      0|       874|      page|      main|1696371064|           0|    null|
|      0|       874|     event|      main|1696372696|        main|    null|
|      0|       874|     event|      main|1696373564|        main|    null|
|      0|       874|      page|    rabota|1696374894|        main|    null|
|      0|       874|     event|    rabota|1696377393|      rabota|    null|
|      0|       874|      page|    online|1696378229|      rabota|    null|
|      0|       874|     event|    online|1696378928|      online|    null|
|      0|       879|      page|      main|1696768416|           0|    null|
|      0|       879|      page|    online|1696768738|        main|    null|
|      0|       879|      page|   tariffs|1696768973|      online|    null|
|      0|   

4. We filter the last table, removing errors and rows where current and previous events are the same.

In [10]:
se.sql("""
SELECT DISTINCT
    user_id,
    session_id,
    event_page,
    timestamp
FROM (
    SELECT DISTINCT
        c.user_id,
        c.session_id,
        c.event_type,
        c.event_page,
        c.timestamp,
        c.lagged_event,
        e.is_error
    FROM (
        SELECT DISTINCT
            *,
            LAG(c.event_page, 1, 0) OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp ASC) AS lagged_event
        FROM clicks AS c
        ORDER BY c.user_id, c.session_id, c.timestamp) 
    AS c
    LEFT JOIN (
        SELECT DISTINCT user_id, session_id, timestamp, 1 AS is_error
        FROM clicks
        WHERE event_type LIKE "%error%"
        ) AS e
    ON
        c.user_id = e.user_id
        AND c.session_id = e.session_id
        AND c.timestamp >= e.timestamp
    ORDER BY c.user_id, c.session_id, c.timestamp) AS tbl1
WHERE 
    is_error IS NULL
    AND NOT event_page = lagged_event
    OR lagged_event = 0
ORDER BY tbl1.user_id, tbl1.session_id, tbl1.timestamp;
""").show(30, False)

+-------+----------+----------+----------+
|user_id|session_id|event_page|timestamp |
+-------+----------+----------+----------+
|0      |874       |main      |1696371064|
|0      |874       |rabota    |1696374894|
|0      |874       |online    |1696378229|
|0      |879       |main      |1696768416|
|0      |879       |online    |1696768738|
|0      |879       |tariffs   |1696768973|
|0      |879       |online    |1696769277|
|0      |879       |main      |1696773185|
|0      |879       |internet  |1696774086|
|0      |879       |online    |1696776502|
|0      |885       |main      |1697348270|
|0      |888       |main      |1697564550|
|0      |898       |main      |1697594437|
|0      |898       |news      |1697596341|
|0      |898       |tariffs   |1697598240|
|0      |898       |rabota    |1697600131|
|0      |898       |bonus     |1697605450|
|0      |898       |tariffs   |1697611362|
|0      |898       |bonus     |1697612492|
|0      |898       |internet  |1697614807|
|0      |89

5. We take the last table, create a route for each user-session combination.

In [11]:
se.sql("""
SELECT 
    user_id, 
    session_id, 
    concat_ws('-', sort_array(collect_list(struct(timestamp, event_page))).event_page) as route
FROM (
    SELECT DISTINCT
        user_id,
        session_id,
        event_page,
        timestamp
    FROM (
        SELECT DISTINCT
            c.user_id,
            c.session_id,
            c.event_type,
            c.event_page,
            c.timestamp,
            c.lagged_event,
            e.is_error
        FROM (
            SELECT DISTINCT
                *,
                LAG(c.event_page, 1, 0) OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp ASC) AS lagged_event
            FROM clicks AS c
            ORDER BY c.user_id, c.session_id, c.timestamp) 
        AS c
        LEFT JOIN (
            SELECT DISTINCT user_id, session_id, timestamp, 1 AS is_error
            FROM clicks
            WHERE event_type LIKE "%error%"
            ) AS e
        ON
            c.user_id = e.user_id
            AND c.session_id = e.session_id
            AND c.timestamp >= e.timestamp
        ORDER BY c.user_id, c.session_id, c.timestamp) AS tbl1
    WHERE 
        is_error IS NULL
        AND NOT event_page = lagged_event
        OR lagged_event = 0) AS t1
GROUP BY user_id, session_id
ORDER BY user_id, session_id
""").show(30, False)#.toPandas().to_csv('mycsv_sql.csv')

+-------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id|session_id|route                                                                                                                                                                                                                                                      |
+-------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0      |874       |main-rabota-online                                                                                                                                               

6. Final table

In [12]:
se.sql("""
SELECT
    route, 
    COUNT(*) AS count
FROM (
    SELECT 
        user_id, 
        session_id, 
        concat_ws('-', sort_array(collect_list(struct(timestamp, event_page))).event_page) as route
    FROM (
        SELECT DISTINCT
            user_id,
            session_id,
            event_page,
            timestamp
        FROM (
            SELECT DISTINCT
                c.user_id,
                c.session_id,
                c.event_type,
                c.event_page,
                c.timestamp,
                c.lagged_event,
                e.is_error
            FROM (
                SELECT DISTINCT
                    *,
                    LAG(c.event_page, 1, 0) OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp ASC) AS lagged_event
                FROM clicks AS c
                ORDER BY c.user_id, c.session_id, c.timestamp) 
            AS c
            LEFT JOIN (
                SELECT DISTINCT user_id, session_id, timestamp, 1 AS is_error
                FROM clicks
                WHERE event_type LIKE "%error%"
                ) AS e
            ON
                c.user_id = e.user_id
                AND c.session_id = e.session_id
                AND c.timestamp >= e.timestamp
            ORDER BY c.user_id, c.session_id, c.timestamp) AS tbl1
        WHERE 
            is_error IS NULL
            AND NOT event_page = lagged_event
            OR lagged_event = 0) AS t1
    GROUP BY user_id, session_id
    ORDER BY user_id, session_id
    ) AS t2
GROUP BY route
ORDER BY count DESC;
""").show(30, False)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8185 |
|main-archive         |1112 |
|main-rabota          |1047 |
|main-internet        |896  |
|main-bonus           |870  |
|main-news            |769  |
|main-tariffs         |677  |
|main-online          |587  |
|main-vklad           |518  |
|main-rabota-archive  |170  |
|main-archive-rabota  |167  |
|main-bonus-archive   |143  |
|main-rabota-bonus    |138  |
|main-bonus-rabota    |136  |
|main-news-rabota     |135  |
|main-archive-internet|132  |
|main-rabota-news     |130  |
|main-internet-rabota |129  |
|main-archive-news    |126  |
|main-rabota-internet |124  |
|main-internet-archive|123  |
|main-archive-bonus   |117  |
|main-internet-bonus  |115  |
|main-tariffs-internet|114  |
|main-news-archive    |113  |
|main-news-internet   |109  |
|main-archive-tariffs |104  |
|main-internet-news   |103  |
|main-tariffs-archive |103  |
|main-rabota-main     |94   |
+---------

Finally, save the output:

In [13]:
se.sql("""
SELECT
    route, 
    COUNT(*) AS count
FROM (
    SELECT 
        user_id, 
        session_id, 
        concat_ws('-', sort_array(collect_list(struct(timestamp, event_page))).event_page) as route
    FROM (
        SELECT DISTINCT
            user_id,
            session_id,
            event_page,
            timestamp
        FROM (
            SELECT DISTINCT
                c.user_id,
                c.session_id,
                c.event_type,
                c.event_page,
                c.timestamp,
                c.lagged_event,
                e.is_error
            FROM (
                SELECT DISTINCT
                    *,
                    LAG(c.event_page, 1, 0) OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp ASC) AS lagged_event
                FROM clicks AS c
                ORDER BY c.user_id, c.session_id, c.timestamp) 
            AS c
            LEFT JOIN (
                SELECT DISTINCT user_id, session_id, timestamp, 1 AS is_error
                FROM clicks
                WHERE event_type LIKE "%error%"
                ) AS e
            ON
                c.user_id = e.user_id
                AND c.session_id = e.session_id
                AND c.timestamp >= e.timestamp
            ORDER BY c.user_id, c.session_id, c.timestamp) AS tbl1
        WHERE 
            is_error IS NULL
            AND NOT event_page = lagged_event
            OR lagged_event = 0) AS t1
    GROUP BY user_id, session_id
    ORDER BY user_id, session_id
    ) AS t2
GROUP BY route
ORDER BY count DESC
LIMIT 30;
""").toPandas().to_csv('output_sql.txt', sep='\t', index=False)

## RDD Solution

In this solution, we need to create serveral mappers for processing batches of data grouped by different keys.

In [14]:
clicks_rdd = sc.textFile("hdfs:/data/clickstream.csv")

In [15]:
# mapper to create tree structure from table

def mapper(x):
    lst = x.split("\t")
    return [(lst[1], lst[0]), [lst[4], [lst[2], lst[3], lst[2].find("error")]]]

In [16]:
# test

mapper('562\t507\tpage\tbruUzJzerrorByY\t1695584127')

[('507', '562'), ['1695584127', ['page', 'bruUzJzerrorByY', -1]]]

In [17]:
# mapper to mark lines after error: 1 if error, -1 if not

def mapper_error(x):
    
    found_error = False
    last_route = None
    
    for event in x[1]:
        
        if found_error:
            event[1][2] = 1
        elif event[1][2] > -1:
            found_error = True
            event[1][2] = 1
        
        if last_route is None:
            last_route = event[1][1]
            event[1].append(None)
        else:
            event[1].append(last_route)
            last_route = event[1][1]
            
    return x

In [18]:
# test

z = [('486', '3121'),
  [['1695591308', ['page', 'main', -1]],
   ['1695591324', ['uockKXVberrornBMUlkiIu', 'main', 8]],
   ['1695591330', ['event', 'main', -1]],
   ['1695591340', ['event', 'main', -1]],
   ['1695591360', ['page', 'vklad', -1]],
   ['1695591373', ['event', 'vklad', -1]],
   ['1695591420', ['event', 'vklad', -1]],
   ['1695591420', ['event', 'vklad', -1]],
   ['1695591441', ['page', 'archive', -1]],
   ['1695591459', ['event', 'archive', -1]],
   ['1695591463', ['event', 'archive', -1]],
   ['1695591473', ['page', 'digital', -1]],
   ['1695591483', ['event', 'digital', -1]]]]
z = mapper_error(z)
z

[('486', '3121'),
 [['1695591308', ['page', 'main', -1, None]],
  ['1695591324', ['uockKXVberrornBMUlkiIu', 'main', 1, 'main']],
  ['1695591330', ['event', 'main', 1, 'main']],
  ['1695591340', ['event', 'main', 1, 'main']],
  ['1695591360', ['page', 'vklad', 1, 'main']],
  ['1695591373', ['event', 'vklad', 1, 'vklad']],
  ['1695591420', ['event', 'vklad', 1, 'vklad']],
  ['1695591420', ['event', 'vklad', 1, 'vklad']],
  ['1695591441', ['page', 'archive', 1, 'vklad']],
  ['1695591459', ['event', 'archive', 1, 'archive']],
  ['1695591463', ['event', 'archive', 1, 'archive']],
  ['1695591473', ['page', 'digital', 1, 'archive']],
  ['1695591483', ['event', 'digital', 1, 'digital']]]]

In [19]:
# mapper to aggregate data grom group by to separate lines

def demapper(x):
    
    out = []
    
    for elem in x[1]:
        out.append([x[0], elem])
        
    return out

In [20]:
# test

z = [('60', '3204'),
  [['1695589018', ['page', 'main', -1]],
   ['1695589071', ['page', 'internet', -1]]]]
demapper(z)

[[('60', '3204'), ['1695589018', ['page', 'main', -1]]],
 [('60', '3204'), ['1695589071', ['page', 'internet', -1]]]]

In [21]:
# mapper to obtain routes from series of records

def get_route(x):
    
    routes = []
    
    for event in x[1]:
        routes.append(event[1][1])
        
    return [x[0], "-".join(routes)]
    

In [22]:
# test

z = [('914', '2211'),
  [['1698218843', ['event', 'tariffs', -1, None]],
   ['1698218920', ['page', 'bonus', -1, 'tariffs']],
   ['1698219836', ['page', 'rabota', -1, 'bonus']],
   ['1698219957', ['page', 'vklad', -1, 'rabota']],
   ['1698171414', ['page', 'main', -1, 'vklad']]]]

get_route(z)

[('914', '2211'), 'tariffs-bonus-rabota-vklad-main']

Final result:

In [23]:
(
    clicks_rdd
    .map(mapper)
    .filter(lambda x: x[0][0] != "session_id")
    .groupByKey()
    .map(lambda x: [x[0], list(x[1])])
    .map(mapper_error)
    .flatMap(demapper)
    .filter(lambda x: x[1][1][2] == -1 and x[1][1][1] != x[1][1][3])
    .groupByKey()
    .map(lambda x: [x[0], list(x[1])])
    .map(get_route)
    .flatMap(lambda x: [(x[1], 1)])
    .groupByKey()
    .map(lambda x: (x[0], len(x[1])))
    .sortBy(lambda x: -x[1])
    .take(30)
)

[('main', 8185),
 ('main-archive', 1113),
 ('main-rabota', 1047),
 ('main-internet', 897),
 ('main-bonus', 870),
 ('main-news', 769),
 ('main-tariffs', 677),
 ('main-online', 587),
 ('main-vklad', 518),
 ('main-rabota-archive', 170),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 143),
 ('main-rabota-bonus', 139),
 ('main-news-rabota', 135),
 ('main-bonus-rabota', 135),
 ('main-archive-internet', 132),
 ('main-rabota-news', 130),
 ('main-internet-rabota', 129),
 ('main-archive-news', 126),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-internet-bonus', 115),
 ('main-news-archive', 113),
 ('main-tariffs-internet', 113),
 ('main-news-internet', 109),
 ('main-archive-tariffs', 104),
 ('main-internet-news', 103),
 ('main-tariffs-archive', 103),
 ('main-rabota-main', 94)]

Save to file:

In [24]:
(
    clicks_rdd
    .map(mapper)
    .filter(lambda x: x[0][0] != "session_id")
    .groupByKey()
    .map(lambda x: [x[0], list(x[1])])
    .map(mapper_error)
    .flatMap(demapper)
    .filter(lambda x: x[1][1][2] == -1 and x[1][1][1] != x[1][1][3])
    .groupByKey()
    .map(lambda x: [x[0], list(x[1])])
    .map(get_route)
    .flatMap(lambda x: [(x[1], 1)])
    .groupByKey()
    .map(lambda x: (x[0], len(x[1])))
    .sortBy(lambda x: -x[1])
    .toDF()
    .toPandas()[:30]
    .to_csv('output_rdd.csv', sep='\t', index=False)
)

## DF Solution

In [25]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col, udf
from pyspark.sql import functions as F
from pyspark.sql.functions import desc
from pyspark.sql.types import StringType

windowSpec  = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

list2str = udf(lambda z: "-".join(z), StringType())   

In [26]:
error_df = (
    clicks
    .withColumn("is_error", col("event_type").contains("error"))
    .sort("user_id", "session_id", "timestamp")
    .filter("is_error == True")
    .withColumnRenamed("timestamp", "right_timestamp")
    .withColumnRenamed("user_id", "right_user_id")
    .withColumnRenamed("session_id", "right_session_id")
    .select("right_user_id", "right_session_id", "right_timestamp", "is_error")
)

error_df.show(10)

+-------------+----------------+---------------+--------+
|right_user_id|right_session_id|right_timestamp|is_error|
+-------------+----------------+---------------+--------+
|            0|             879|     1696777359|    true|
|            0|             898|     1697629831|    true|
|            0|             901|     1699003239|    true|
|            0|             912|     1700596574|    true|
|            0|             922|     1700724457|    true|
|            1|            1013|     1699237817|    true|
|            1|            1030|     1700376100|    true|
|            1|             972|     1696964963|    true|
|            1|             981|     1697304119|    true|
|            1|             981|     1697312938|    true|
+-------------+----------------+---------------+--------+
only showing top 10 rows



In [27]:
(
    clicks
    .withColumn("lag", lag("event_page",1).over(windowSpec))
    .sort("user_id", "session_id", "timestamp")
    # .join(error_df, (clicks.user_id == error_df.right_user_id) & 
    #       (clicks.session_id == error_df.right_session_id) & 
    #       ((clicks.timestamp < error_df.right_timestamp)),
    #      how = "left")
    # .filter("lag <> event_page")
    # .select(clicks.user_id, clicks.session_id,
    #         clicks.event_page, clicks.timestamp)
    # .sort(clicks.user_id, clicks.session_id, clicks.timestamp)
    # .groupby('user_id', 'session_id')
    # .agg(F.collect_list("event_page").alias("route"))
    # .groupby("route")
    # .count()
    # .sort(desc("count"))
    .show(50, False)
)

+-------+----------+----------+----------+----------+--------+
|user_id|session_id|event_type|event_page|timestamp |lag     |
+-------+----------+----------+----------+----------+--------+
|0      |874       |page      |main      |1696371064|null    |
|0      |874       |event     |main      |1696372696|main    |
|0      |874       |event     |main      |1696373564|main    |
|0      |874       |page      |rabota    |1696374894|main    |
|0      |874       |event     |rabota    |1696377393|rabota  |
|0      |874       |page      |online    |1696378229|rabota  |
|0      |874       |event     |online    |1696378928|online  |
|0      |879       |page      |main      |1696768416|null    |
|0      |879       |page      |online    |1696768738|main    |
|0      |879       |page      |tariffs   |1696768973|online  |
|0      |879       |page      |online    |1696769277|tariffs |
|0      |879       |event     |online    |1696770139|online  |
|0      |879       |event     |online    |1696771080|on

Final table:

In [28]:
(
    clicks
    .withColumn("lag", lag("event_page",1).over(windowSpec))
    .join(error_df, (clicks.user_id == error_df.right_user_id) & 
          (clicks.session_id == error_df.right_session_id) & 
          ((clicks.timestamp >= error_df.right_timestamp)),
         how = "left")
    .filter("is_error is null")
    .filter("lag <> event_page OR lag is null")
    # .sort("user_id", "session_id", "timestamp")
    .select(clicks.user_id, clicks.session_id,
            clicks.event_page, clicks.timestamp)
    # .sort(clicks.user_id, clicks.session_id, clicks.timestamp)
    .groupby('user_id', 'session_id')
    .agg(F.collect_list("event_page").alias("route"))
    .withColumn("route", list2str(col("route")))
    .groupby("route")
    .count()
    .sort(desc("count"))
    .show(30, False)
)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8184 |
|main-archive         |1111 |
|main-rabota          |1045 |
|main-internet        |896  |
|main-bonus           |870  |
|main-news            |769  |
|main-tariffs         |677  |
|main-online          |587  |
|main-vklad           |518  |
|main-rabota-archive  |170  |
|main-archive-rabota  |167  |
|main-bonus-archive   |143  |
|main-rabota-bonus    |139  |
|main-news-rabota     |135  |
|main-bonus-rabota    |135  |
|main-archive-internet|132  |
|main-rabota-news     |130  |
|main-internet-rabota |129  |
|main-archive-news    |126  |
|main-rabota-internet |124  |
|main-internet-archive|123  |
|main-archive-bonus   |117  |
|main-internet-bonus  |115  |
|main-tariffs-internet|114  |
|main-news-archive    |113  |
|main-news-internet   |109  |
|main-archive-tariffs |104  |
|main-internet-news   |103  |
|main-tariffs-archive |103  |
|main-rabota-main     |94   |
+---------

Save to file:

In [29]:
(
    clicks
    .withColumn("lag", lag("event_page",1).over(windowSpec))
    .join(error_df, (clicks.user_id == error_df.right_user_id) & 
          (clicks.session_id == error_df.right_session_id) & 
          ((clicks.timestamp >= error_df.right_timestamp)),
         how = "left")
    .filter("is_error is null")
    .filter("lag <> event_page OR lag is null")
    # .sort("user_id", "session_id", "timestamp")
    .select(clicks.user_id, clicks.session_id,
            clicks.event_page, clicks.timestamp)
    # .sort(clicks.user_id, clicks.session_id, clicks.timestamp)
    .groupby('user_id', 'session_id')
    .agg(F.collect_list("event_page").alias("route"))
    .withColumn("route", list2str(col("route")))
    .groupby("route")
    .count()
    .sort(desc("count"))
    .toPandas()[:30]
    .to_csv('output_DF.csv', sep='\t', index=False)
)